## Описание

Практика следующих тем:
- Парсинг
- Визуализация 
- Географическая визуализация

Предметная область данных:
- Фигурное катание

## Импорты

In [ ]:
pip install beautifulsoup4 

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install lxml


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Получение данных

В качестве источника взят сайт с официальными результатами соревнований по фигруному катанию в новой системе оценивания

Общий вид страницы выглядит следующим образом:

https://skatingscores.com/2425/wc/sr/women/i/short/usa/alysa_liu/

https://skatingscores.com/сезон/чемпионат/возрастная категория/дисциплина/i/программа/страна/участник/

Соберем информацию для чемпионатов мира за все года с новой системой оценивания в взрослом женском одиночном катании. Для этого нужно получить список всех имен участниц



In [3]:
# Общая страница с результатами коротной программы сезона 24-25
url = 'https://skatingscores.com/2425/wc/sr/women/i/short/'
# Начало ссылки для участницы
base_link = 'https://skatingscores.com/2425/wc/sr/women/i/short/'
# Проверяем доступ 
page = requests.get(url)
# print(page.status_code)
# Загружаем данные
soup = BeautifulSoup(page.text, "html.parser")

In [ ]:
## Нам нужно будет пройтись по всем годам


In [ ]:
# Получаем все ссылки
df_total_score = pd.DataFrame()
for link in soup.find_all('a'):
    link_ = link.get('href')
    # Фильтруем ссылки участниц от ссылок на судей и прочих возможных по шаблону
    if '/re/2425/wc/sr/women/i/short' in link_:
        country_girl = link_.split('/')[-3]
        girl = link_.split('/')[-2]
        # Собираем ссылку
        girl_link = base_link+'/'+country_girl+'/'+girl+'/'
        # print(girl_link)
        page_girl = requests.get(girl_link)
        # print(page_girl.status_code)    
        soup_girl = BeautifulSoup(page_girl.text, "html.parser")
        time.sleep(3)
        # break
        # Получаем все таблицы со страницы
        ### Здесь много таблиц, так как в каждой представлена своя информация по различным оценкам иих состовляющим
        tables = soup_girl.find_all('table')  
        # Берем нужные нам параметры из каждой таблицы
        # Здесь самая общая информация без подробных оценок за элементы
        df_1 = pd.read_html(str(tables[0]))[0].tail(1).rename(columns={1: 'Name', 2: 'Country', 4: 'Dedaction', 5: 'Total'})
        df_2 = pd.read_html(str(tables[1]))[0]    
        df_2 = df_2.drop(columns=[i for i  in range(6)]+[15]).rename(columns=df_2.iloc[0]).tail(1)
        df_3 = pd.read_html(str(tables[3]))[0][[0,1]].T
        df_3 = df_3.rename(columns=df_3.iloc[0]).tail(1)
        df_4 = pd.read_html(str(tables[10]))[0]
        df_4 = df_4[[15]].rename(columns={15: 'Component'}).reset_index()
        df_4['index'] = 1
        df_4 = df_4.set_index(['index'])
        df_first = df_1.join(df_2)
        df_second = df_3.join(df_4)
        df_all = df_first.join(df_second)
        df_all = df_all.drop([0], axis=1)
        df_total_score = pd.concat([df_total_score,df_all])  
# # Убираем лишние цифры
df_total_score = df_total_score.replace(to_replace ='\d+\s', value = '', regex = True)
df_total_score['Name'] = df_total_score['Name'].replace(to_replace ='\d+', value = '', regex = True)
# Убираем лишние буквы
df_total_score = df_total_score.replace(to_replace ='SB', value = '', regex = True) 
df_total_score = df_total_score.replace(to_replace ='PB', value = '', regex = True) 
df_total_score = df_total_score[['Name','Country','Dedaction','Total','ISR 🇮🇱','SVK 🇸🇰','CZE 🇨🇿','FRA 🇫🇷','NOR 🇳🇴','SUI 🇨🇭','JPN 🇯🇵','ITA 🇮🇹','ROU 🇷🇴','BV','GOE','Jumps','Spins','TES','Component']]


In [122]:
df_total_score

,Name,Country,Dedaction,Total,ISR 🇮🇱,SVK 🇸🇰,CZE 🇨🇿,FRA 🇫🇷,NOR 🇳🇴,SUI 🇨🇭,JPN 🇯🇵,ITA 🇮🇹,ROU 🇷🇴,BV,GOE,Jumps,Spins,TES,Component
1,Alysa LIU,USA,0.00,74.58,77.23,75.71,73.06,72.85,74.68,73.65,73.92,75.45,74.65,32.59,8.21,22.76,12.75,40.80,33.78
1,Mone CHIBA,JPN,0.00,73.44,72.79,73.43,71.87,74.15,74.57,75.04,71.86,75.10,74.04,31.93,7.41,22.27,12.59,39.34,34.10
1,Isabeau LEVITO,USA,0.00,73.33,74.18,75.19,73.28,68.65,70.59,75.11,74.12,74.73,71.42,32.59,6.68,21.50,12.37,39.27,34.06
1,Wakaba HIGUCHI,JPN,0.00,72.10,77.62,69.52,71.46,65.98,73.39,71.40,72.11,73.73,72.71,32.33,7.38,22.71,11.48,39.71,32.39
1,Kaori SAKAMOTO,JPN,0.00,71.03,72.33,68.51,69.59,69.94,71.47,71.76,71.47,71.44,70.87,28.00,7.99,19.73,11.64,35.99,35.04
1,Madeline SCHIZAS,CAN,0.00,69.18,69.13,68.18,70.23,68.19,67.20,70.00,69.22,70.70,68.77,31.89,6.56,22.40,11.09,38.45,30.73
1,Haein LEE,KOR,0.00,67.79,69.95,62.66,68.77,65.59,67.56,68.62,67.89,69.63,68.32,32.33,4.56,20.45,11.20,36.89,30.90
1,Nina PINZARRONE,BEL,0.00,67.74,68.87,66.63,68.91,66.96,68.62,68.64,64.87,66.96,67.74,32.09,4.11,19.32,11.87,36.20,31.54
1,Amber GLENN,USA,-1.00,67.65,69.19,61.78,68.82,64.77,67.68,68.18,66.83,69.68,69.39,34.89,1.02,20.95,10.72,35.91,32.74
1,Kimmy REPOND,SUI,0.00,67.42,69.54,62.33,67.54,68.60,64.93,70.87,66.52,68.73,66.13,30.89,5.58,20.74,11.58,36.47,30.95


### Проходимся по странице

In [31]:
# Получаем все таблицы со страницы
tables = soup_girl.find_all('table')

In [ ]:
# Из первой таблицы оставляем имя, страну, дедакшен(штраф) и итоговую сумму баллов
df_1 = pd.read_html(str(tables[0]))[0].tail(1).rename(columns={1: 'Name', 2: 'Country', 4: 'Dedaction', 5: 'Total'})
df_2 = pd.read_html(str(tables[1]))[0]    
df_2 = df_2.drop(columns=[i for i  in range(6)]+[15]).rename(columns=df_2.iloc[0]).tail(1)
df_3 = pd.read_html(str(tables[3]))[0][[0,1]].T
df_3 = df_3.rename(columns=df_3.iloc[0]).tail(1)
df_4 = pd.read_html(str(tables[10]))[0]
df_4 = df_4[[15]].rename(columns={15: 'Component'}).reset_index()
df_4['index'] = 1
df_4 = df_4.set_index(['index'])
df_first = df_1.join(df_2)
df_second = df_3.join(df_4)
df_all = df_first.join(df_second)
df_all = df_all.drop([0], axis=1)
df_all

In [79]:
## Все таблицы надо привести к формату для анализа, оставив только нужные показатели

# В итоге должно получится две таблицы - в разрезе общих результатов и подробно по элементам

# Из первой таблицы оставляем имя, страну, дедакшен(штраф) и итоговую сумму баллов
df_1 = pd.read_html(str(tables[0]))[0].tail(1).rename(columns={1: 'Name', 2: 'Country', 4: 'Dedaction', 5: 'Total'})
# df_1 = df_1.drop([3], axis=1).reset_index()
# df_1 = df_1.drop(['index'], axis=1)
df_1


C:\Users\пользователь\AppData\Local\Temp\ipykernel_11032\780115722.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_1 = pd.read_html(str(tables[0]))[0].tail(1).rename(columns={1: 'Name', 2: 'Country', 4: 'Dedaction', 5: 'Total'})


,0,Name,Country,3,Dedaction,Total
1,33,Anastasia GOZHVA76,UKR,4,-2.00,37.54


In [82]:
# Во второй TSS по судьям
df_2 = pd.read_html(str(tables[1]))[0]    
df_2 = df_2.drop(columns=[i for i  in range(6)]+[15]).rename(columns=df_2.iloc[0]).tail(1)
# Убираем лишние цифры
# df_2 = df_2.replace(to_replace ='\d{1}\s', value = '', regex = True)
# df_2 = df_2.drop(['index'], axis=1)   
df_2      


C:\Users\пользователь\AppData\Local\Temp\ipykernel_11032\459431234.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_2 = pd.read_html(str(tables[1]))[0]


,ISR 🇮🇱,SVK 🇸🇰,CZE 🇨🇿,FRA 🇫🇷,NOR 🇳🇴,SUI 🇨🇭,JPN 🇯🇵,ITA 🇮🇹,ROU 🇷🇴
1,33 36.51,33 37.10,33 39.52,33 36.19,32 39.40,33 35.48,33 39.06,33 36.22,32 37.83


In [83]:
## Из этой таблицы возьмем техническую разбивку - BV, GOE, Jumps, Spins, TES
df_3 = pd.read_html(str(tables[3]))[0][[0,1]].T
df_3 = df_3.rename(columns=df_3.iloc[0]).tail(1)
# Убираем лишние цифры
# df_3  = df_3.replace(to_replace ='\d{1}\s', value = '', regex = True)
# Убираем лишние буквы
# df_3  = df_3.replace(to_replace ='SB', value = '', regex = True)
# df_3 = df_3.drop(['BVSB 2 32.59'], axis=1).reset_index()
# df_3 = df_3.drop(['index'], axis=1)
# df_3 = df_3[['BV','GOE','Jumps','Spins','TES']]
df_3

C:\Users\пользователь\AppData\Local\Temp\ipykernel_11032\4161011731.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_3 = pd.read_html(str(tables[3]))[0][[0,1]].T


,BV 29 24.95,BV,GOE,Jumps,Spins,TES
1,GOE 32 -4.90,29 24.95,32 -4.90,33 7.03,24 9.72,31 20.05


In [97]:
df_4 = pd.read_html(str(tables[10]))[0]
# df_4  = df_4.replace(to_replace ='\d{1}\s', value = '', regex = True)
# df_4  = df_4.replace(to_replace ='PB', value = '', regex = True)
# df_4 = df_4[[15]]
df_4 = df_4[[15]].rename(columns={15: 'Component'}).reset_index()
df_4['index'] = 1
# df_4 = df_4.drop(['index'], axis=1)
df_4 = df_4.set_index(['index'])

C:\Users\пользователь\AppData\Local\Temp\ipykernel_11032\4108294425.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_4 = pd.read_html(str(tables[10]))[0]


In [96]:
df_first = df_1.join(df_2)
df_second = df_3.join(df_4)
df_all = df_first.join(df_second)
df_all = df_all.drop([0], axis=1)
df_all

,Name,Country,3,Dedaction,Total,ISR 🇮🇱,SVK 🇸🇰,CZE 🇨🇿,FRA 🇫🇷,NOR 🇳🇴,...,JPN 🇯🇵,ITA 🇮🇹,ROU 🇷🇴,BV 29 24.95,BV,GOE,Jumps,Spins,TES,Component
1,Anastasia GOZHVA76,UKR,4,-2.00,37.54,33 36.51,33 37.10,33 39.52,33 36.19,32 39.40,...,33 39.06,33 36.22,32 37.83,GOE 32 -4.90,29 24.95,32 -4.90,33 7.03,24 9.72,31 20.05,33 19.49


In [ ]:
# Элементы подробно

# ## Третья - оценки за элементы по технике
# df_3 = pd.read_html(str(tables[2]))[0]

# pd.read_html(str(tables[9]))[0]